decoder transformer层的linear2层（dtl2）替换为近似矩阵乘法, 32\*32\*128, 128\*64, 32\*32\*64

In [13]:
import numpy as np
import os
import matmul as mm
import math_util as mu
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [14]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [15]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [16]:
linear_name = 'dtl2'
feedback_bits = 256
ncodebooks = 128 # max:128
ncentroids = 256
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 200 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [17]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'dx_linear2in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'dx_linear2_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'dx_linear2out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'dx_linear2in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    featurepath_test = 'dx_linear2out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dirs")


weightpath = 'dx_linear2_w_f256.npy'
biaspath = 'dx_linear2_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [18]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    15335.366763896935
learn_multisplits(): returning loss:  116.17975596431643
learn_multisplits(): initial loss:    2729.5847010158745
learn_multisplits(): returning loss:  12.348652828484774
learn_multisplits(): initial loss:    34684.49062377929
learn_multisplits(): returning loss:  140.88686467846568
learn_multisplits(): initial loss:    5804.05514449954
learn_multisplits(): returning loss:  51.06512192636728
learn_multisplits(): initial loss:    4465.688923904597
learn_multisplits(): returning loss:  29.832055100705475
learn_multisplits(): initial loss:    332277.4404678556
learn_multisplits(): returning loss:  185.87064290046692
learn_multisplits(): initial loss:    13935.287383412127
learn_multisplits(): returning loss:  107.04461818165146
learn_multisplits(): initial loss:    3783.001805448234
learn_multisplits(): returning loss:  25.572614498902112
learn_multisplits(): initia

In [19]:
# type(est3)

In [20]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [21]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-2.8218367  -0.9621016  -0.8115635  ... -0.73980373  1.0360615
  -0.6223679 ]
 [-2.8218367   1.0418123  -0.8115635  ... -2.7437177   1.0360615
   1.381546  ]
 [-0.8179226  -2.9660156   1.1923504  ... -0.73980373 -2.9717665
   3.38546   ]
 ...
 [-2.8218367  -0.9621016  -0.8115635  ... -2.7437177   3.0399754
   1.381546  ]
 [-2.8218367  -0.9621016   1.1923504  ... -2.7437177   1.0360615
   1.381546  ]
 [-2.8218367  -0.9621016  -0.8115635  ... -2.7437177   1.0360615
   1.381546  ]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
